In [1]:
import BioSimSpace as bss

/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
solvent = "TIP3P"
box_edges = 20
box_type = "orthorhombic"
angstrom = bss.Units.Length.angstrom

In [3]:
protein_path = "../inputs/protein/"
ligand_path = "../inputs/ligands/"
ligand_name_1 = "ligand_8_xtal.sdf"
ligand_name_2 = "docked_10.sdf"
ligand_1 = bss.IO.readMolecules(ligand_path+ligand_name_1)[0]
ligand_2 = bss.IO.readMolecules(ligand_path+ligand_name_2)[0]
ligand_1_params = bss.Parameters.gaff2(ligand_1, net_charge=-1).getMolecule()
ligand_2_params = bss.Parameters.gaff2(ligand_2, net_charge=-1).getMolecule()


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [6]:
ligand_params = [ligand_1_params, ligand_2_params]
ligand_names = [ligand_name_1, ligand_name_2]
for i in range(2):
    box_min, box_max = ligand_params[i].getAxisAlignedBoundingBox()
    box_size = [y - x for x, y in zip(box_min, box_max)]
    box_sizes = [x + box_edges * angstrom for x in box_size]
    box, angles = bss.Box.cubic(max(box_sizes))
    print(f"Box size set to {max(box_sizes)}")
    print("Solvating ligand.")
    ligand_params_solvated = bss.Solvent.solvate(solvent, 
                                                 molecule=ligand_params[i],
                                                 box=box,
                                                 angles=angles)
    print("Solvation done. Saving ligand.")
    save_name = ligand_names[i].strip(".sdf").split("_")[1]
    bss.IO.saveMolecules(f"../inputs/ligands/ligand_{save_name}_solv", ligand_params_solvated, ["PRM7", "RST7"])                          
    bss.IO.saveMolecules(f"../inputs/ligands/ligand_{save_name}_solv", ligand_params_solvated, ["Gro87", "GroTop"])                                            

Box size set to 27.1000 A
Solvating ligand.
Solvation done. Saving ligand.
Box size set to 27.6280 A
Solvating ligand.
Solvation done. Saving ligand.


In [7]:
protein = bss.IO.readMolecules(["../inputs/protein/prot_water.rst7",
                                "../inputs/protein/prot_water.prm7"])[0]
for i in range(2):
    system = ligand_params[i] + protein
    box_min_sys, box_max_sys = system.getAxisAlignedBoundingBox()
    box_size_sys = [y - x for x, y in zip(box_min_sys, box_max_sys)]
    box_sizes_sys = [x + box_edges * angstrom for x in box_size_sys]
    box, angles = bss.Box.cubic(max(box_sizes_sys))
    print(f"Box size set to {max(box_sizes_sys)}")
    print("Solvating system")
    system_solvated = bss.Solvent.solvate(solvent,
                                          molecule=system,
                                          box=box,
                                          angles=angles)
    save_name = ligand_names[i].strip(".sdf").split("_")[1]
    bss.IO.saveMolecules(f"../inputs/protein/system_{save_name}_solv", system_solvated, ["PRM7", "RST7"])
    bss.IO.saveMolecules(f"../inputs/protein/system_{save_name}_solv", system_solvated, ["Gro87", "GroTop"])

Box size set to 64.3260 A
Solvating system
Box size set to 64.3260 A
Solvating system
